# Miscellaneous

### autograd

In [ ]:
try:
    from autograd import grad  # Only required for optionally verifying gradients while training
    from autograd import numpy as grad_np
    AUTOGRAD_PRESENT = True
except ImportError:
    AUTOGRAD_PRESENT = False

#@staticmethod
def _loss_fn(matrix, regularization_coeff=1.0):
    """Computes loss value.
    Parameters
    ----------
    matrix : numpy.array
        Array containing vectors for u, v and negative samples, of shape (2 + negative_size, dim).
    regularization_coeff : float, optional
        Coefficient to use for l2-regularization
    Returns
    -------
    float
        Computed loss value.
    Warnings
    --------
    Only used for autograd gradients, since autograd requires a specific function signature.
    """
    vector_u = matrix[0]
    vectors_v = matrix[1:]
    euclidean_dists = grad_np.linalg.norm(vector_u - vectors_v, axis=1)
    norm = grad_np.linalg.norm(vector_u)
    all_norms = grad_np.linalg.norm(vectors_v, axis=1)
    poincare_dists = grad_np.arccosh(
        1 + 2 * (
            (euclidean_dists ** 2) / ((1 - norm ** 2) * (1 - all_norms ** 2))
        )
    )
    exp_negative_distances = grad_np.exp(-poincare_dists)
    regularization_term = regularization_coeff * grad_np.linalg.norm(vectors_v[0]) ** 2
    return -grad_np.log(exp_negative_distances[0] / (exp_negative_distances.sum())) + regularization_term

In [ ]:
u = relations[0][0]
v = relations[0][1]
#grad_loss(u, v)

theta = embedding_dict[u]
x = embedding_dict[v]
negatives = negative_sample(relations, u, num_negative)
neg_v = np.array([embedding_dict[sample[1]] for sample in negatives])
vector_u = theta
vectors_v = np.vstack((x,neg_v))
matrix = np.vstack((vector_u, vectors_v))
matrix

autograd_loss = grad(_loss_fn)
autograd_loss(matrix)

grad_loss(u, v)